In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import numpy as np
from torchtext import data

import random
import math
import time

from transformers import T5Model,T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
tokenizer.vocab_size

32100

In [4]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [6]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [8]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [10]:
SRC = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

TRG = Field(batch_first = True,
          use_vocab = False,
          tokenize = tokenize_and_cut,
          preprocessing = tokenizer.convert_tokens_to_ids,
          init_token = init_token_idx,
          eos_token = eos_token_idx,
          pad_token = pad_token_idx,
          unk_token = unk_token_idx)

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
fields = [('src', SRC), ('trg', TRG)]

In [12]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'news_train.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(1234))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [13]:
print(len(train_data.examples))
print(len(valid_data.examples))

86765
1771


In [14]:
print(vars(train_data.examples[2000]))

{'src': [662, 151, 130, 4792, 57, 3, 9, 1634, 53, 2412, 298, 1119, 12, 2269, 3, 9, 14421, 1463, 16, 3, 547, 107, 52, 9, 7, 3939, 30, 3, 189, 3589, 1135, 6, 3, 76, 17, 2046, 3, 5319, 1395, 107, 2095, 243, 5, 386, 13, 8, 8926, 130, 45, 8, 337, 384, 5, 8, 5415, 808, 286, 1084, 3, 547, 107, 52, 9, 7, 14421, 2478, 11, 3778, 7076, 15, 4283, 12, 43, 118, 2953, 57, 13809, 20735, 11, 2714, 12435, 6, 8, 2095, 974, 5], 'trg': [2412, 3154, 147, 662, 151, 16, 95, 31, 7, 3, 547, 107, 52, 9, 7, 3939]}


In [15]:
src_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['src'])
trg_tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[2000])['trg'])

print(src_tokens)
print(trg_tokens)

['▁four', '▁people', '▁were', '▁killed', '▁by', '▁', 'a', '▁speed', 'ing', '▁train', '▁while', '▁trying', '▁to', '▁cross', '▁', 'a', '▁railway', '▁track', '▁in', '▁', 'hat', 'h', 'r', 'a', 's', '▁district', '▁on', '▁', 'th', 'urs', 'day', ',', '▁', 'u', 't', 'tar', '▁', 'pra', 'des', 'h', '▁police', '▁said', '.', '▁three', '▁of', '▁the', '▁victims', '▁were', '▁from', '▁the', '▁same', '▁family', '.', '▁the', '▁incident', '▁took', '▁place', '▁near', '▁', 'hat', 'h', 'r', 'a', 's', '▁railway', '▁station', '▁and', '▁prima', '▁faci', 'e', '▁appeared', '▁to', '▁have', '▁been', '▁caused', '▁by', '▁dense', '▁fog', '▁and', '▁poor', '▁visibility', ',', '▁the', '▁police', '▁added', '.']
['▁train', '▁runs', '▁over', '▁four', '▁people', '▁in', '▁up', "'", 's', '▁', 'hat', 'h', 'r', 'a', 's', '▁district']


In [16]:
device = torch.device('cuda')

BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits(
                                 (train_data, valid_data), 
                                 batch_size = BATCH_SIZE,
                                 device = device,
                                 sort_key=lambda x: len(x.src))

/home/gauranga/anaconda3/envs/pt/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [17]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(self.t5.config.to_dict()['d_model'],
                             self.t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src,decoder_input_ids=trg) 
        
        output = self.out(embedded[0])
        
        return output

In [18]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [20]:
# CONVERT ALL MODEL WEIGHTS AND BIASES TO HALF PRECISION
# MODEL SIZE WILL REDUCE
model = model.half()

In [21]:
model.load_state_dict(torch.load('t5_summ_model_3.pt'))

<All keys matched successfully>

In [22]:
def translate_sentence2(sentence, eval_model, device, max_len = 50):
    
    eval_model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = eval_model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break

    return trg_indexes[1:]

In [23]:
idxs = random.sample(range(0, len(valid_data.examples)), 20)
for i in idxs:
    src = vars(valid_data.examples[i])['src']
    #trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence2(src, model, device)

    print(f"SRC : {''.join(tokenizer.convert_ids_to_tokens(src))}")
    #print(f"TRG : {''.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PRED : {''.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁previous▁indian▁presidents▁had▁various▁professions,▁the▁most▁being▁teachers▁and▁lawyers,▁before▁being▁elected.▁india's▁first▁president,▁rajendra▁prasad,▁was▁a▁teacher▁and▁lawyer▁before▁becoming▁president.▁kr▁narayanan▁and▁current▁president▁pranab▁mukherjee▁were▁journalists▁before.▁further,▁apj▁abdul▁kalam▁was▁a▁scientist▁and▁administrator▁at▁defence▁research▁and▁development▁organisation▁(drdo).
PRED : ▁previous▁indian▁presidents▁had▁different▁professions▁before▁becoming▁prez</s>

SRC : ▁cricket▁advisory▁committee▁(cac)▁member▁sourav▁ganguly▁has▁said▁zaheer▁khan▁will▁spend▁150▁days▁a▁year▁with▁the▁indian▁team▁as▁overseas▁bowling▁consultant.▁zaheer▁reportedly▁did▁not▁wish▁to▁commit▁for▁more▁than▁100▁days▁annually,▁but▁gave▁in▁to▁the▁cricket▁advisory▁committee's▁insistence.▁zaheer▁is▁expected▁to▁accompany▁the▁team▁for▁the▁sri▁lanka▁tour.
PRED : ▁zaheer▁to▁spend▁150▁days▁a▁year▁with▁team▁as▁overseas▁bowling▁consultant</s>

SRC : ▁speaking▁at▁the▁india-korea▁business▁summit▁on▁tuesda

In [24]:
def return_summary(txt):
    txt_tokens = tokenizer.tokenize(txt)
    txt_ids = tokenizer.convert_tokens_to_ids(txt_tokens)
    pred = translate_sentence2(txt_ids, model, device)
    pred_tokens = tokenizer.convert_ids_to_tokens(pred)
    
    return ''.join(pred_tokens)

In [25]:
TEXT = "the united nations confirmed tuesday that un secretary general kofi annan will hold a third round of talks with iraqi foreign minister naji sabri in vienna in early july ."

In [26]:
return_summary(TEXT)

'▁un▁confirms▁kofi▁annan▁to▁hold▁3rd▁round▁of▁talks▁with▁iraqi</s>'